In [1]:
import numpy as np
import pandas as pd
import random
import sys
import os
from tqdm import tqdm
sys.path.append("./tools/")
import commontools
import ucTools
import funclib
import time
import joblib

import train
import test

%load_ext autoreload
%autoreload 2

In [30]:
uctools = ucTools.ucTools('172.16.25.20')
cnx_ecdb = uctools.db_conn()

# 1. 加载序列序列文件
file_1515 = './tasks/task3/data/ecoli1515/1515testset_with_unirep.tsv'
data_1515 = pd.read_csv(file_1515, sep='\t', index_col=0)

colname = [
    ['id',
     'ec_number',
     'seq',
     'isenzyme',
     'isMultiFunctional',
     'blattner_id',
     'blattner_ec_number'] + ['f' + str(i) for i in range(1, 1901)]]

data_1515.columns = np.array(colname)[0]

file_ref = './data/preprocess/enzyme_train.fasta'
blast_res_file = './results/1515/balastres.tsv'

# 2. 获取序列比对结果
blast_res = test.getblast(querydata=data_1515, refdata=file_ref, blastres_file=blast_res_file)

# 3.获取酶-非酶预测结果
isEnzyme_model = './model/isenzyme.model'
isEnzyme_pred, isEnzyme_pred_prob = test.get_isEnzymeRes(querydata=data_1515.iloc[:, 8:1909], model_file=isEnzyme_model)

# 4.获取Slice预测结果
slice_pred = test.get_slice_res(data_1515.iloc[:, 7:1909], './model', './results/1515/slice_results.txt')

# 5. 获取最终结果
file_label_ec = './data/dict_label_ec.npy'
final_pred = test.get_final_results(ori_data=data_1515,
                               blast_data=blast_res,
                               isenzyme_data=isEnzyme_pred,
                               slice_data=slice_pred,
                               cnx=cnx_ecdb,
                               file_label_ec=file_label_ec
                               )

./slice_predict ./data/preprocess/slice_x_1515.txt ./model ./results/1515/slice_results.txt -o 32 -b 0 -t 32 -q 0


/home/shizhenkun/codebase/BioUniprot/test.py:170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ori_data['isenzyme_slice'] = isenzyme_data


In [31]:
final_pred

,id,ec_number,isenzyme_slice,pident,isenzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast,0,1,...,10,11,12,13,14,15,16,17,18,19
0,P42641,3.6.5.-,1,100.0,True,False,1.0,3.6.5.-,3.6.5.2,1.18.6.1,...,6.2.1.33,6.3.5.4,2.3.1.239,2.3.1.240,6.3.1.15,6.2.1.32,6.2.1.51,2.3.1.111,6.2.1.1,6.2.1.27
1,P0AE22,3.1.3.2,1,100.0,True,False,1.0,3.1.3.2,3.1.3.2,2.5.1.31,...,2.5.1.28,2.4.1.232,4.6.1.13,3.5.1.3,1.13.11.29,3.5.1.28,3.1.16.1,1.17.9.1,3.1.4.41,1.14.99.39
2,P21499,3.1.13.1,1,100.0,True,False,1.0,3.1.13.1,5.2.1.8,3.1.13.1,...,2.1.1.310,2.1.1.202,2.1.1.203,3.2.1.105,2.1.1.167,2.3.1.296,2.1.1.221,3.6.5.5,2.7.11.1,2.4.1.136
3,P39286,3.6.1.-,1,100.0,True,False,1.0,3.6.1.-,2.9.1.3,2.7.4.33,...,2.8.2.24,2.7.4.28,2.7.11.32,2.7.7.1,2.7.4.27,2.8.2.2,6.2.1.46,2.4.1.135,2.8.2.3,2.8.2.23
4,P56100,7.1.1.7,0,100.0,True,False,1.0,7.1.1.7,7.1.1.2,7.1.1.7,...,1.17.4.4,4.3.2.9,1.11.1.11,1.8.3.2,3.1.26.3,2.4.1.132,2.4.1.257,1.1.99.22,3.3.2.11,3.2.2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7131,P36837,NaN,0,100.0,False,False,1.0,-,7.2.2.10,2.7.1.202,...,2.4.1.17,3.4.16.5,2.7.7.74,2.7.8.34,2.1.1.86,7.1.1.9,2.7.7.41,2.7.1.66,4.1.1.111,6.3.3.3
7132,P77304,NaN,0,100.0,False,False,1.0,-,7.2.2.10,7.2.2.8,...,1.8.7.3,3.1.3.4,1.21.99.4,2.1.1.86,1.8.98.6,7.2.2.13,2.7.13.3,1.2.5.3,7.2.2.4,4.1.1.111
7133,P27248,2.1.2.10,1,100.0,True,False,1.0,2.1.2.10,2.1.2.10,2.7.2.11,...,2.8.3.9,2.5.1.94,4.1.2.42,4.2.1.33,2.3.1.277,2.4.2.54,4.4.1.24,4.2.1.55,2.7.7.68,2.4.2.57
7134,P0AC88,4.2.1.47,1,100.0,True,False,1.0,4.2.1.47,4.2.1.47,2.4.1.62,...,3.7.1.12,2.7.1.157,4.1.1.105,4.1.99.5,3.6.1.64,5.1.3.15,1.14.13.25,5.1.3.26,2.1.1.179,1.1.1.308


In [32]:
final_pred[final_pred.isenzyme_slice == final_pred.isenzyme_blast]

,id,ec_number,isenzyme_slice,pident,isenzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast,0,1,...,10,11,12,13,14,15,16,17,18,19
0,P42641,3.6.5.-,1,100.0,True,False,1.0,3.6.5.-,3.6.5.2,1.18.6.1,...,6.2.1.33,6.3.5.4,2.3.1.239,2.3.1.240,6.3.1.15,6.2.1.32,6.2.1.51,2.3.1.111,6.2.1.1,6.2.1.27
1,P0AE22,3.1.3.2,1,100.0,True,False,1.0,3.1.3.2,3.1.3.2,2.5.1.31,...,2.5.1.28,2.4.1.232,4.6.1.13,3.5.1.3,1.13.11.29,3.5.1.28,3.1.16.1,1.17.9.1,3.1.4.41,1.14.99.39
2,P21499,3.1.13.1,1,100.0,True,False,1.0,3.1.13.1,5.2.1.8,3.1.13.1,...,2.1.1.310,2.1.1.202,2.1.1.203,3.2.1.105,2.1.1.167,2.3.1.296,2.1.1.221,3.6.5.5,2.7.11.1,2.4.1.136
3,P39286,3.6.1.-,1,100.0,True,False,1.0,3.6.1.-,2.9.1.3,2.7.4.33,...,2.8.2.24,2.7.4.28,2.7.11.32,2.7.7.1,2.7.4.27,2.8.2.2,6.2.1.46,2.4.1.135,2.8.2.3,2.8.2.23
5,P23874,2.7.11.1,1,100.0,True,False,1.0,2.7.11.1,6.1.1.14,2.5.1.18,...,5.4.99.30,2.5.1.10,2.5.1.1,5.3.1.9,5.4.99.43,3.8.1.10,2.1.1.200,1.14.13.69,3.5.99.2,1.14.13.172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7131,P36837,NaN,0,100.0,False,False,1.0,-,7.2.2.10,2.7.1.202,...,2.4.1.17,3.4.16.5,2.7.7.74,2.7.8.34,2.1.1.86,7.1.1.9,2.7.7.41,2.7.1.66,4.1.1.111,6.3.3.3
7132,P77304,NaN,0,100.0,False,False,1.0,-,7.2.2.10,7.2.2.8,...,1.8.7.3,3.1.3.4,1.21.99.4,2.1.1.86,1.8.98.6,7.2.2.13,2.7.13.3,1.2.5.3,7.2.2.4,4.1.1.111
7133,P27248,2.1.2.10,1,100.0,True,False,1.0,2.1.2.10,2.1.2.10,2.7.2.11,...,2.8.3.9,2.5.1.94,4.1.2.42,4.2.1.33,2.3.1.277,2.4.2.54,4.4.1.24,4.2.1.55,2.7.7.68,2.4.2.57
7134,P0AC88,4.2.1.47,1,100.0,True,False,1.0,4.2.1.47,4.2.1.47,2.4.1.62,...,3.7.1.12,2.7.1.157,4.1.1.105,4.1.99.5,3.6.1.64,5.1.3.15,1.14.13.25,5.1.3.26,2.1.1.179,1.1.1.308


In [11]:
slice_pred_top, slice_pred_top_pb = test.sort_slice_results(slice_pred)
slice_pred_top = test.slice_label2ec(slice_pred=slice_pred_top, file_label_ec='./data/dict_label_ec.npy')

In [20]:
slice_id_out = pd.concat([data_1515.iloc[:,0], slice_pred_top], axis=1)

In [22]:
slice_id_out.drop_duplicates(subset=None, keep='first', inplace=True)